# Setup

In [83]:
from pathlib import Path
import numpy as np
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import nltk

pd.options.display.float_format = '{:20,.2f}'.format

In [84]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/abbit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/abbit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
ROOT_DIR = Path() / ".."
DATA_DIR =  ROOT_DIR / "data"

In [3]:
x_train_df = pd.read_csv(DATA_DIR / "X_train.csv")
x_test_df = pd.read_csv(DATA_DIR / "X_test.csv")
y_train_df = pd.read_csv(DATA_DIR / "Y_train.csv")
df = pd.merge(x_train_df, y_train_df, on="id").drop(columns=["id"])
df['created_at'] = pd.to_datetime(df['created_at'])
df['published_at'] = pd.to_datetime(df['published_at'])

# Utils

In [4]:
def describe(df, count=False, missing_info=False):
    desc = df.describe(include='all').T
    if not count:
        desc = desc.drop('count', axis=1)
    if missing_info:
        desc['missing'] = df.isnull().sum()
        desc['missing %'] = desc['missing'] / len(df) * 100
        cols = desc.columns.tolist()
        cols = cols[-2:] + cols[:-2]
        desc = desc[cols]
    print(f"Total rows: {len(df)}")
    return desc

In [5]:
# Load spaCy model for Russian
nlp = spacy.load("ru_core_news_sm")

def is_contains_fio(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PER":
            return True

    return False

# Explore

**Описание призников датасета**:
- id (int) - Идентификатор вакансии. С помощью этого ключа можно смержить X и y;
- name (text) - Наименование вакансии;
- has_test (bool) - Указывает на наличие испытательного срока;
- response_letter_required (bool) - Указывает на необходимость наличия рекомендательного письма
- salary_from (number | None) - Нижний порог заработной платы;
- salary_currency (string) - Краткое наименование валюты, в которой указана вилка ЗП;
- salary_gross (bool | None) - Указывает на то, была ли рассчитана заработная плата с учётом налога или нет:
- published_at (datetime) - Дата и время публикации вакансии;
- created_at (datetime) - Дата и время создания вакансии
- employer_name (text) - Наименование ЮР лица\ИП разместивший вакансию
- description (text) - описание вакансии
- area_id (int) - идентификатор города
- area_name: (string) - название города

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27934 entries, 0 to 27933
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   name                      27934 non-null  object             
 1   has_test                  27934 non-null  bool               
 2   response_letter_required  27934 non-null  bool               
 3   salary_from               23902 non-null  float64            
 4   salary_currency           27934 non-null  object             
 5   salary_gross              27786 non-null  object             
 6   published_at              27934 non-null  datetime64[ns, UTC]
 7   created_at                27934 non-null  datetime64[ns, UTC]
 8   employer_name             27934 non-null  object             
 9   description               27933 non-null  object             
 10  area_id                   27934 non-null  int64              
 11  area_name      

In [7]:
df_with_area_id_as_str = df.copy()
df_with_area_id_as_str["area_id"] = df_with_area_id_as_str["area_id"].astype(str)
describe(df_with_area_id_as_str.drop(['salary_from', 'salary_to', 'published_at', 'created_at'], axis=1), missing_info=True)

Total rows: 27934


,missing,missing %,unique,top,freq
name,0,0.00,16395,Менеджер по продажам,337
has_test,0,0.00,2,False,27227
response_letter_required,0,0.00,2,False,26847
salary_currency,0,0.00,1,RUR,27934
salary_gross,148,0.53,2,False,18726
employer_name,0,0.00,13318,Пятёрочка,927
description,1,0.00,23681,<p><strong>Обязанности:</strong></p><ul><li>Об...,624
area_id,0,0.00,156,1,20512
area_name,0,0.00,156,Москва,20512


In [8]:
describe(df[['published_at', 'created_at']], missing_info=True)

Total rows: 27934


,missing,missing %,mean,min,25%,50%,75%,max
published_at,0,0.00,2020-10-29 17:27:34.720054272+00:00,2020-03-12 06:12:50+00:00,2020-06-12 20:52:03.500000+00:00,2020-08-19 18:35:53.500000+00:00,2021-03-18 06:10:38.249999872+00:00,2021-12-01 14:21:11+00:00
created_at,0,0.00,2020-10-29 17:27:34.720054272+00:00,2020-03-12 06:12:50+00:00,2020-06-12 20:52:03.500000+00:00,2020-08-19 18:35:53.500000+00:00,2021-03-18 06:10:38.249999872+00:00,2021-12-01 14:21:11+00:00


In [9]:
describe(df[['salary_from', 'salary_to']], missing_info=True)

Total rows: 27934


,missing,missing %,mean,std,min,25%,50%,75%,max
salary_from,4032,14.43,"64,179.46","55,530.79",1.00,"33,500.00","50,000.00","80,000.00","3,400,000.00"
salary_to,0,0.00,"104,418.22","105,299.26",10.00,"45,000.00","70,000.00","140,000.00","5,000,000.00"


## `created_at` и `published_at`

In [10]:
time_difference = df['created_at'] == df['published_at']
print(f"created_at and published_at is always the same: {time_difference.all()}")

created_at and published_at is always the same: True


In [11]:
datetime_df = df[['created_at']].copy()
datetime_df['date'] = datetime_df['created_at'].dt.date
datetime_df['time'] = datetime_df['created_at'].dt.time
datetime_df['year'] = datetime_df['created_at'].dt.year
datetime_df['quarter'] = datetime_df['created_at'].dt.quarter
datetime_df['month'] = datetime_df['created_at'].dt.month
datetime_df['day'] = datetime_df['created_at'].dt.day
datetime_df['hour'] = datetime_df['created_at'].dt.hour
datetime_df['weekday'] = datetime_df['created_at'].dt.weekday
datetime_df['is_weekend'] = datetime_df['weekday'].isin([5, 6])
datetime_df['is_month_start'] = datetime_df['created_at'].dt.is_month_start
datetime_df['is_month_end'] = datetime_df['created_at'].dt.is_month_end
datetime_df['is_quarter_start'] = datetime_df['created_at'].dt.is_quarter_start
datetime_df['is_quarter_end'] = datetime_df['created_at'].dt.is_quarter_end
datetime_df['is_year_start'] = datetime_df['created_at'].dt.is_year_start
datetime_df['is_year_end'] = datetime_df['created_at'].dt.is_year_end
describe(datetime_df.drop(['created_at'], axis=1))

Total rows: 27934


,unique,top,freq,mean,std,min,25%,50%,75%,max
date,606,2020-09-11,215,NaN,NaN,NaN,NaN,NaN,NaN,NaN
time,20630,15:05:51,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
year,NaN,NaN,NaN,"2,020.33",0.47,"2,020.00","2,020.00","2,020.00","2,021.00","2,021.00"
quarter,NaN,NaN,NaN,2.47,0.89,1.00,2.00,3.00,3.00,4.00
month,NaN,NaN,NaN,6.46,2.61,1.00,4.00,7.00,8.00,12.00
day,NaN,NaN,NaN,15.85,8.73,1.00,8.00,16.00,23.00,31.00
hour,NaN,NaN,NaN,12.05,4.97,0.00,8.00,12.00,15.00,23.00
weekday,NaN,NaN,NaN,2.69,1.96,0.00,1.00,3.00,4.00,6.00
is_weekend,2,False,21842,NaN,NaN,NaN,NaN,NaN,NaN,NaN
is_month_start,2,False,27089,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
print(f"min created time: {df['created_at'].min()}")
print(f"max created time: {df['created_at'].max()}")

min created time: 2020-03-12 06:12:50+00:00
max created time: 2021-12-01 14:21:11+00:00


In [13]:
datetime_df['date'].value_counts().sort_index().hvplot.line(title="Number of job offers per day")

:Curve   [date]   (count)

In [14]:
datetime_df['year'].value_counts().sort_index().hvplot.bar(title="Number of job offers per year")

:Bars   [year]   (count)

In [15]:
datetime_df['month'].value_counts().sort_index().hvplot.bar(title='Number of job offers per month')

:Bars   [month]   (count)

In [16]:
datetime_df['weekday'].value_counts().sort_index().hvplot.bar(title="Number of job posts by weekday")

:Bars   [weekday]   (count)

In [17]:
datetime_df['hour'].value_counts().sort_index().hvplot.bar(title="Number of job posts by hour")

:Bars   [hour]   (count)

## `area_id` и `area_name`

In [18]:
area_mapping = df.set_index('area_name')['area_id'].to_dict()
is_same = df['area_name'].map(area_mapping) == df['area_id']
print(f"area_name mapped to area_id is always the same: {is_same.all()}")

area_name mapped to area_id is always the same: True


In [19]:
# group `area_name` with values less than 100 to `other`
area_df = df[['area_name']].copy()
area_name_count = area_df['area_name'].value_counts()
area_name_count = area_name_count[area_name_count < 100]
area_name_count = area_name_count.index.tolist()
area_df['area_name'] = area_df['area_name'].replace(area_name_count, 'other')
area_df['area_name'].astype(str).value_counts().hvplot.bar(title="Number of job posts by area", invert=True, flip_yaxis=True)

:Bars   [area_name]   (count)

## `employer_name`

In [20]:
employers = df['employer_name']
# replace ё with е
employers = employers.str.replace('ё', 'е')
print(f"Number of unique employers: {employers.nunique()}")

Number of unique employers: 13317


In [21]:
# get employers with `ИП` in name
ip_employers = employers[employers.str.contains('ИП')]
ip_employers

8          Мебель-Мастер (ИП Зверев Олег Владимирович)
33       Energy Fashion (ИП Кулинич Ксения Алексеевна)
147            TeamHorizont (ИП Русских Денис Юрьевич)
406                                        ИП Киреенко
423            FitCurves (ИП Левадняя Елена Сергеевна)
                             ...                      
27348           PomaBrush (ИП Полищук Тарас Сергеевич)
27405            Ferlenz (ИП Цыганов Денис Николаевич)
27496            Ferlenz (ИП Цыганов Денис Николаевич)
27795                                      ИП-ОПетрова
27861               СДЭК (ИП Садовых Софья Михайловна)
Name: employer_name, Length: 309, dtype: object

In [22]:
employers_with_fio = employers[employers.apply(is_contains_fio)]
employers_with_fio

8          Мебель-Мастер (ИП Зверев Олег Владимирович)
14                       Епифанов Михаил Александрович
26                                        Дадаев Медиа
31                           Платова Ангелина Игоревна
33       Energy Fashion (ИП Кулинич Ксения Алексеевна)
                             ...                      
27912                      Родионов Алексей Алексеевич
27916                  Каширин Константин Владимирович
27923                  Харчевников Александр Андреевич
27929                        Балтик Граунд Сервисес РУ
27931                    Баскаков Дмитрий Владимирович
Name: employer_name, Length: 4870, dtype: object

In [23]:
employers_with_fio[employers_with_fio.apply(lambda x: x.split().__len__() == 3)]

14         Епифанов Михаил Александрович
31             Платова Ангелина Игоревна
46       Огородников Максим Владимирович
67               Павлова Ольга Андреевна
78                      ТК Мейджик Транс
                      ...               
27901        Чаусов Сергей Александрович
27912        Родионов Алексей Алексеевич
27916    Каширин Константин Владимирович
27923    Харчевников Александр Андреевич
27931      Баскаков Дмитрий Владимирович
Name: employer_name, Length: 1932, dtype: object

## `name`

In [95]:
def process_name(name):
    name = nltk.word_tokenize(name, language='russian')
    name = [word.lower().strip() for word in name]
    name = [word for word in name if len(word) > 2]
    return name

def is_ascii_only(name):
    return all(ord(c) < 128 for c in name)

In [92]:
process_name("ООО «АвтоГЕРМЕС»")

['ооо', 'автогермес']

In [106]:
# Split text in 'name' column by whitespace and concatenate into one list
name_words = pd.Series(df['name'].map(process_name).sum())
# Count the most frequent words
name_word_counts = name_words.value_counts()

In [108]:
# Display the most frequent words
name_word_counts.head(50)

менеджер                3926
продажам                1516
developer               1470
москва                  1453
разработчик             1433
продавец-кассир         1347
специалист              1328
помощник                 993
удаленно                 978
работе                   834
администратор            831
программист              803
отдела                   776
аналитик                 730
инженер                  728
python                   715
оператор                 654
manager                  560
клиентами                550
ведущий                  546
senior                   532
middle                   484
продавец-консультант     483
санкт-петербург          466
junior                   442
юрист                    429
frontend                 404
php                      404
руководитель             403
продаж                   398
улица                    380
рекрутер                 370
backend                  350
ассистент                346
закупкам      

In [104]:
name_words[name_words.map(is_ascii_only)].value_counts().head(50)

developer        1470
python            715
manager           560
senior            532
middle            484
junior            442
php               404
frontend          404
backend           350
product           306
engineer          282
java              274
react             241
data              206
remote            173
devops            155
front-end         146
ios               138
c++               138
scientist         117
lead              111
android           110
middle/senior     107
analyst           106
project            92
recruiter          92
javascript         89
golang             88
marketing          81
digital            81
.net               77
fullstack          76
unity              73
django             70
web                68
laravel            67
full               66
stack              66
team               65
ruby               63
vue.js             59
b2b                59
back-end           58
full-stack         57
software           54
linux     

## `salary_to`

In [24]:
salary_to = df['salary_to']
describe(pd.DataFrame({'salary_to': salary_to}))

Total rows: 27934


,mean,std,min,25%,50%,75%,max
salary_to,"104,418.22","105,299.26",10.00,"45,000.00","70,000.00","140,000.00","5,000,000.00"


In [25]:
missing_salary_from = df[df['salary_from'].isnull()]
describe(missing_salary_from[['salary_to']])

Total rows: 4032


,mean,std,min,25%,50%,75%,max
salary_to,"105,922.21","98,473.35",23.00,"45,000.00","70,000.00","150,000.00","3,000,000.00"


In [26]:
missing_salary_from['salary_to'].value_counts().sort_index().hvplot.bar()

:Bars   [salary_to]   (count)

In [27]:
missing_salary_from['salary_to'].hvplot.violin()

:Violin   (salary_to)

In [28]:
# remove outliers
salaries_without_outliers = salary_to[salary_to.between(salary_to.quantile(.01), salary_to.quantile(.99))]
salaries_without_outliers.hvplot.violin(title="Salary without outliers")

:Violin   (salary_to)

In [29]:
describe(pd.DataFrame({'salary': salaries_without_outliers}))

Total rows: 27523


,mean,std,min,25%,50%,75%,max
salary,"99,939.92","75,455.45","15,000.00","45,000.00","70,000.00","130,000.00","400,000.00"


In [30]:
print(f"outliers count: {len(salary_to) - len(salaries_without_outliers)}")

outliers count: 411


In [43]:
# Define the bin edges
salary_group_step = 10000
bin_edges = np.arange(10000, salaries_without_outliers.max() + salary_group_step, salary_group_step)

# Group the salaries without outliers
salary_groups = pd.cut(salaries_without_outliers, bins=bin_edges)

# Count the number of salaries in each group
salary_counts = salary_groups.value_counts().sort_index()

# Display the grouped salaries
salary_counts.index = salary_counts.index.astype(str)

salary_counts.hvplot.bar(title="Salary distribution", invert=True, flip_yaxis=True)


:Bars   [salary_to]   (count)

In [55]:
df_for_correlation = df[['has_test', 'response_letter_required', 'salary_from', 'created_at', 'area_id', 'salary_to']]
df_for_correlation.corr(method='spearman').hvplot.heatmap(title="Correlation matrix")

:HeatMap   [columns,index]   (value)